In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline
import seaborn as sns
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split, KFold
from sklearn import metrics
import xgboost as xgb
import time

In [28]:
data = pd.read_csv('input/train.csv')

In [3]:
test_data = pd.read_csv('input/test.csv')

In [4]:
test_data.shape

(125546, 131)

In [5]:
data[data.cat2 == 'B']['loss'].mean()

3800.104170741578

In [6]:
mu = data['loss'].mean()
sigma = data['loss'].std()


In [7]:
data.describe()

,id,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,loss
count,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000
mean,294135.982561,0.493861,0.507188,0.498918,0.491812,0.487428,0.490945,0.484970,0.486437,0.485506,0.498066,0.493511,0.493150,0.493138,0.495717,3037.337686
std,169336.084867,0.187640,0.207202,0.202105,0.211292,0.209027,0.205273,0.178450,0.199370,0.181660,0.185877,0.209737,0.209427,0.212777,0.222488,2904.086186
min,1.000000,0.000016,0.001149,0.002634,0.176921,0.281143,0.012683,0.069503,0.236880,0.000080,0.000000,0.035321,0.036232,0.000228,0.179722,0.670000
25%,147748.250000,0.346090,0.358319,0.336963,0.327354,0.281143,0.336105,0.350175,0.312800,0.358970,0.364580,0.310961,0.311661,0.315758,0.294610,1204.460000
50%,294539.500000,0.475784,0.555782,0.527991,0.452887,0.422268,0.440945,0.438285,0.441060,0.441450,0.461190,0.457203,0.462286,0.363547,0.407403,2115.570000
75%,440680.500000,0.623912,0.681761,0.634224,0.652072,0.643315,0.655021,0.591045,0.623580,0.566820,0.614590,0.678924,0.675759,0.689974,0.724623,3864.045000
max,587633.000000,0.984975,0.862654,0.944251,0.954297,0.983674,0.997162,1.000000,0.980200,0.995400,0.994980,0.998742,0.998484,0.988494,0.844848,121012.250000


# """remove outliers""" 
# """convert to numerics (Categorical)"""
# """scale the continuuos features"""
# """groupby for categorical binary features -EDA """

In [8]:
# remove outliers in the loss part 
data = data[(data['loss']- mu ) / sigma < 3]

In [8]:
data.describe()

,id,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,loss
count,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000
mean,294135.982561,0.493861,0.507188,0.498918,0.491812,0.487428,0.490945,0.484970,0.486437,0.485506,0.498066,0.493511,0.493150,0.493138,0.495717,3037.337686
std,169336.084867,0.187640,0.207202,0.202105,0.211292,0.209027,0.205273,0.178450,0.199370,0.181660,0.185877,0.209737,0.209427,0.212777,0.222488,2904.086186
min,1.000000,0.000016,0.001149,0.002634,0.176921,0.281143,0.012683,0.069503,0.236880,0.000080,0.000000,0.035321,0.036232,0.000228,0.179722,0.670000
25%,147748.250000,0.346090,0.358319,0.336963,0.327354,0.281143,0.336105,0.350175,0.312800,0.358970,0.364580,0.310961,0.311661,0.315758,0.294610,1204.460000
50%,294539.500000,0.475784,0.555782,0.527991,0.452887,0.422268,0.440945,0.438285,0.441060,0.441450,0.461190,0.457203,0.462286,0.363547,0.407403,2115.570000
75%,440680.500000,0.623912,0.681761,0.634224,0.652072,0.643315,0.655021,0.591045,0.623580,0.566820,0.614590,0.678924,0.675759,0.689974,0.724623,3864.045000
max,587633.000000,0.984975,0.862654,0.944251,0.954297,0.983674,0.997162,1.000000,0.980200,0.995400,0.994980,0.998742,0.998484,0.988494,0.844848,121012.250000


In [9]:
columns = data.columns.values

In [10]:
enc = preprocessing.LabelEncoder()
for col in columns:
    if col.startswith("cat"):
        data[col] = enc.fit_transform(data[col])

In [11]:
data = data.drop('id', axis = 1)

In [12]:
enc = preprocessing.LabelEncoder()
for col in columns:
    if col.startswith("cat"):
        test_data[col] = enc.fit_transform(test_data[col])

In [13]:
data.head()

,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10,...,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,loss
0,0,1,0,1,0,0,0,0,1,0,...,0.718367,0.335060,0.30260,0.67135,0.83510,0.569745,0.594646,0.822493,0.714843,2213.18
1,0,1,0,0,0,0,0,0,1,1,...,0.438917,0.436585,0.60087,0.35127,0.43919,0.338312,0.366307,0.611431,0.304496,1283.60
2,0,1,0,0,1,0,0,0,1,1,...,0.289648,0.315545,0.27320,0.26076,0.32446,0.381398,0.373424,0.195709,0.774425,3005.09
3,1,1,0,1,0,0,0,0,1,0,...,0.440945,0.391128,0.31796,0.32128,0.44467,0.327915,0.321570,0.605077,0.602642,939.85
4,0,1,0,1,0,0,0,0,1,1,...,0.178193,0.247408,0.24564,0.22089,0.21230,0.204687,0.202213,0.246011,0.432606,2763.85


In [14]:
test_data.head()

,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14
0,4,0,1,0,0,0,0,0,0,1,...,0.281143,0.466591,0.317681,0.61229,0.34365,0.38016,0.377724,0.369858,0.704052,0.392562
1,6,0,1,0,1,0,0,0,0,1,...,0.836443,0.482425,0.443760,0.71330,0.51890,0.60401,0.689039,0.675759,0.453468,0.208045
2,9,0,1,0,1,1,0,1,0,1,...,0.718531,0.212308,0.325779,0.29758,0.34365,0.30529,0.245410,0.241676,0.258586,0.297232
3,12,0,0,0,0,1,0,0,0,0,...,0.397069,0.369930,0.342355,0.40028,0.33237,0.31480,0.348867,0.341872,0.592264,0.555955
4,15,1,0,0,0,0,1,0,0,0,...,0.302678,0.398862,0.391833,0.23688,0.43731,0.50556,0.359572,0.352251,0.301535,0.825823


In [20]:
df_train_labels = data['loss']
df_train_data = data.drop('loss', axis=1)
df_test_data = test_data

In [21]:
df_test_data = df_test_data.drop('id', axis=1)

In [22]:
df_train_data.head()

,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14
0,0,1,0,1,0,0,0,0,1,0,...,0.310061,0.718367,0.335060,0.30260,0.67135,0.83510,0.569745,0.594646,0.822493,0.714843
1,0,1,0,0,0,0,0,0,1,1,...,0.885834,0.438917,0.436585,0.60087,0.35127,0.43919,0.338312,0.366307,0.611431,0.304496
2,0,1,0,0,1,0,0,0,1,1,...,0.397069,0.289648,0.315545,0.27320,0.26076,0.32446,0.381398,0.373424,0.195709,0.774425
3,1,1,0,1,0,0,0,0,1,0,...,0.422268,0.440945,0.391128,0.31796,0.32128,0.44467,0.327915,0.321570,0.605077,0.602642
4,0,1,0,1,0,0,0,0,1,1,...,0.704268,0.178193,0.247408,0.24564,0.22089,0.21230,0.204687,0.202213,0.246011,0.432606


In [18]:
df_test_data.describe()

,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14
count,125546.000000,125546.000000,125546.000000,125546.000000,125546.000000,125546.00000,125546.000000,125546.000000,125546.000000,125546.000000,...,125546.000000,125546.000000,125546.000000,125546.000000,125546.000000,125546.000000,125546.000000,125546.000000,125546.000000,125546.000000
mean,0.250506,0.432853,0.054116,0.314785,0.344607,0.29895,0.023896,0.059213,0.398555,0.148169,...,0.487640,0.492188,0.485945,0.487401,0.486015,0.498909,0.494360,0.494033,0.495086,0.495586
std,0.433306,0.495473,0.226247,0.464432,0.475242,0.45780,0.152725,0.236025,0.489603,0.355269,...,0.209119,0.205574,0.178650,0.199549,0.182134,0.185950,0.210399,0.210148,0.213108,0.222613
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.281143,0.012683,0.069503,0.236880,0.000080,0.000000,0.035321,0.036232,0.000228,0.178568
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.281143,0.336105,0.352087,0.317960,0.358970,0.364580,0.310961,0.318249,0.315758,0.294752
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.422268,0.441525,0.438893,0.441060,0.441450,0.466720,0.457203,0.462286,0.363547,0.406104
75%,1.000000,1.000000,0.000000,1.000000,1.000000,1.00000,0.000000,0.000000,1.000000,0.000000,...,0.643315,0.659261,0.591284,0.629180,0.568890,0.619840,0.678924,0.682413,0.689974,0.724791
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,...,0.983107,0.997162,1.000000,0.982800,0.995400,0.994980,0.997830,0.997416,0.988494,0.844814


In [23]:
df_train_data.describe()

,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14
count,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000,...,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000
mean,0.248346,0.433294,0.054827,0.318201,0.342936,0.300688,0.024289,0.058645,0.399303,0.149242,...,0.487428,0.490945,0.484970,0.486437,0.485506,0.498066,0.493511,0.493150,0.493138,0.495717
std,0.432055,0.495532,0.227644,0.465779,0.474692,0.458559,0.153944,0.234961,0.489757,0.356328,...,0.209027,0.205273,0.178450,0.199370,0.181660,0.185877,0.209737,0.209427,0.212777,0.222488
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.281143,0.012683,0.069503,0.236880,0.000080,0.000000,0.035321,0.036232,0.000228,0.179722
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.281143,0.336105,0.350175,0.312800,0.358970,0.364580,0.310961,0.311661,0.315758,0.294610
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.422268,0.440945,0.438285,0.441060,0.441450,0.461190,0.457203,0.462286,0.363547,0.407403
75%,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,...,0.643315,0.655021,0.591045,0.623580,0.566820,0.614590,0.678924,0.675759,0.689974,0.724623
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.983674,0.997162,1.000000,0.980200,0.995400,0.994980,0.998742,0.998484,0.988494,0.844848


In [21]:
print df_train_data.shape, df_test_data.shape

(184794, 130) (125546, 130)


In [25]:
def rmspe_xg(yhat, y):
    y = np.expm1(y.get_label())
    yhat = np.expm1(yhat)
    return "rmspe", rmspe(y, yhat)

def rmspe(y, yhat):
    return np.sqrt(np.mean(((y - yhat)/y) ** 2))

def mae_xg(yhat, y):
    y = np.exp(y.get_label())
    yhat = np.exp(yhat)
    # return "mae", np.mean(np.abs(y - yhat))
    return "mae", metrics.mean_absolute_error(y, yhat)

params = {"objective": "reg:linear",
          "booster" : "gbtree",
          "eta": 0.05,
          "max_depth": 10,
          "subsample": 0.85,
          "colsample_bytree": 0.4,
          "min_child_weight": 10,
          "silent": 1,
          "thread": 1,
          "seed": 1301
          }
num_boost_round = 1200

X_train, X_test, y_train, y_test = train_test_split( df_train_data, df_train_labels, test_size=0.3, random_state=42) 
# y_train_log = np.log1p(y_train)
# y_test_log = np.log1p(y_test)

y_train_log = np.log(y_train)
y_test_log = np.log(y_test)


dtrain = xgb.DMatrix(X_train, y_train_log)
dvalid = xgb.DMatrix(X_test, y_test_log)

watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
"""feval = rmspe_xg,"""
gbm = xgb.train(params, dtrain, num_boost_round, evals=watchlist, early_stopping_rounds=30, feval=mae_xg, verbose_eval=True)

print("Validating")
predicted_labels = gbm.predict(xgb.DMatrix(X_test))
error = metrics.mean_absolute_error(np.exp(y_test_log), np.exp(predicted_labels))
# error = rmspe(y_test, np.expm1(predicted_labels))
print('MAE: {:.6f}'.format(error))

[0]	train-mae:3032.62	eval-mae:3040.48
Multiple eval metrics have been passed: 'eval-mae' will be used for early stopping.

Will train until eval-mae hasn't improved in 30 rounds.
[1]	train-mae:3031.66	eval-mae:3039.52
[2]	train-mae:3030.38	eval-mae:3038.24
[3]	train-mae:3028.71	eval-mae:3036.58
[4]	train-mae:3026.57	eval-mae:3034.44
[5]	train-mae:3023.86	eval-mae:3031.73
[6]	train-mae:3020.47	eval-mae:3028.34
[7]	train-mae:3016.3	eval-mae:3024.17
[8]	train-mae:3011.22	eval-mae:3019.09
[9]	train-mae:3005.13	eval-mae:3013
[10]	train-mae:2997.86	eval-mae:3005.73
[11]	train-mae:2989.33	eval-mae:2997.21
[12]	train-mae:2979.38	eval-mae:2987.26
[13]	train-mae:2967.92	eval-mae:2975.81
[14]	train-mae:2954.87	eval-mae:2962.76
[15]	train-mae:2940.08	eval-mae:2947.98
[16]	train-mae:2923.45	eval-mae:2931.35
[17]	train-mae:2904.99	eval-mae:2912.92
[18]	train-mae:2884.52	eval-mae:2892.46
[19]	train-mae:2862.04	eval-mae:2870.01
[20]	train-mae:2837.69	eval-mae:2845.68
[21]	train-mae:2811.26	eval-mae:2

In [27]:
# np.expm1(
predictions = pd.DataFrame(np.exp(gbm.predict(xgb.DMatrix(df_test_data))))
results = pd.DataFrame();
results['id'] = test_data['id']
results['loss'] = predictions
timestr = time.strftime("%Y%m%d-%H%M%S")
results.to_csv('submission' + timestr + ".csv", index=False)